<a href="https://colab.research.google.com/github/HunterCoders/Prostate-Cancer-Detection-Random-Forest/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/Kaggle_API/kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download soujanyahp/prostate-cancer-dataset

  0% 0.00/2.51k [00:00<?, ?B/s]
100% 2.51k/2.51k [00:00<00:00, 4.06MB/s]


In [ ]:
! unzip /content/prostate-cancer-dataset.zip

Archive:  /content/prostate-cancer-dataset.zip
  inflating: prostate.csv            


In [ ]:
!pip install DataSynthesizer

In [ ]:
import numpy as np
import pandas as pd
import imblearn
import sklearn.pipeline

In [ ]:
file=pd.read_csv('/content/prostate.csv')

In [ ]:
# prompt:
file.head(100)


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,True
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,True
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,True
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,True
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,True
...,...,...,...,...,...,...,...,...,...,...
92,2.830268,3.876396,68,-1.386294,1,1.321756,7,60,4.385147,True
93,3.821004,3.896909,44,-1.386294,1,2.169054,7,40,4.684443,True
94,2.907447,3.396185,52,-1.386294,1,2.463853,7,10,5.143124,False
95,2.882564,3.773910,68,1.558145,1,1.558145,7,80,5.477509,True


In [ ]:
file.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,True
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,True
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,True
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,True
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,True


In [ ]:
file.isnull().sum()
file["train"] = file["train"].astype(int)
file.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,1
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,1
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,1
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,1
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,1


In [ ]:
file.to_csv('/content/finalset.csv')

In [ ]:
filets=pd.read_csv('/content/finalset.csv')
filets.head()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,1
1,1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,1
2,2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,1
3,3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,1
4,4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,1


In [ ]:
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network
# input dataset

input_data='/content/finalset.csv'
# location of two output files
mode = '/content/correlated_attribute_mode'
description_file = f'/content/description.json'
synthetic_data = f'/content/sythetic_data.csv'
candidate_keys={'Pid':True}
categorical_attributes = {"Pid":True,"lcavol":True	,"lweight":True	,"age"	:True	,"lbph":True	,	"svi":True	,	"lcp"	:True	,"gleason"	:True	,"pgg45"	:True	,"lpsa"	:True	,"train":True	}
# An attribute is categorical if its domain size is less than this threshold.
threshold_value = 20
# A parameter in Differential Privacy. It roughly means that removing a row in the input dataset will not
# change the probability of getting the same output more than a multiplicative difference of exp(epsilon).
# Increase epsilon value to reduce the injected noises. Set epsilon=0 to turn off differential privacy.
epsilon = 1
# The maximum number of parents in Bayesian network, i.e., the maximum number of incoming edges.
degree_of_bayesian_network = 2
# Number of tuples generated in synthetic dataset.
num_tuples_to_generate = 10000
#candidate_keys = {'Unnamed': True}
describer = DataDescriber(category_threshold=threshold_value)
describer.describe_dataset_in_correlated_attribute_mode(dataset_file=input_data, epsilon=epsilon, k=degree_of_bayesian_network,attribute_to_is_candidate_key=candidate_keys)
describer.save_dataset_description_to_file(description_file)
display_bayesian_network(describer.bayesian_network)
generator = DataGenerator()
generator.generate_dataset_in_correlated_attribute_mode(num_tuples_to_generate, description_file)
generator.save_synthetic_data(synthetic_data)
# Read both datasets using Pandas.
synthetic_df = pd.read_csv('/content/sythetic_data.csv')
org_df = '/content/finalset.csv'

================ Constructing Bayesian Network (BN) ================
Adding ROOT gleason
Adding attribute lcp
Adding attribute pgg45
Adding attribute svi
Adding attribute lbph
Adding attribute age
Adding attribute lpsa
Adding attribute lweight
Adding attribute train
Adding attribute lcavol
========================== BN constructed ==========================
Constructed Bayesian network:
    lcp     has parents ['gleason'].
    pgg45   has parents ['lcp', 'gleason'].
    svi     has parents ['lcp', 'gleason'].
    lbph    has parents ['lcp', 'gleason'].
    age     has parents ['lcp', 'gleason'].
    lpsa    has parents ['svi', 'lcp'].
    lweight has parents ['age', 'gleason'].
    train   has parents ['lpsa', 'svi'].
    lcavol  has parents ['train', 'lpsa'].


In [ ]:
#file=pd.read_csv('/content/sythetic_data.csv')
file=pd.read_csv('/content/sythetic_data.csv')
file.head()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,0,3.522398,4.156574,59.0,2.174995,1,2.457172,8,35,5.548841,1
1,1,2.466439,3.533182,53.0,2.303647,1,1.416155,8,90,0.832238,1
2,2,-0.204781,4.257076,78.0,2.007251,0,2.501079,8,80,1.885613,0
3,3,-1.280169,3.948632,48.0,-1.149298,1,2.477205,8,75,5.217797,1
4,4,1.627484,2.562938,71.0,1.795352,0,0.902877,7,90,0.249422,0


In [ ]:
X=file.iloc[:,1:-1]
y=file.iloc[:,-1]

In [ ]:
X


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
0,3.522398,4.156574,59.0,2.174995,1,2.457172,8,35,5.548841
1,2.466439,3.533182,53.0,2.303647,1,1.416155,8,90,0.832238
2,-0.204781,4.257076,78.0,2.007251,0,2.501079,8,80,1.885613
3,-1.280169,3.948632,48.0,-1.149298,1,2.477205,8,75,5.217797
4,1.627484,2.562938,71.0,1.795352,0,0.902877,7,90,0.249422
...,...,...,...,...,...,...,...,...,...
9995,2.521690,4.552258,62.0,0.699404,1,-0.150914,8,4,5.444540
9996,-0.384551,4.067641,46.0,1.115856,0,2.329656,7,35,1.260432
9997,2.866007,3.625204,73.0,-0.453329,0,0.179707,9,70,4.939865
9998,3.205736,4.336443,43.0,2.145087,1,1.670624,6,5,2.529629


In [ ]:
X,y

(        lcavol   lweight   age      lbph  svi       lcp  gleason  pgg45  \
 0     3.522398  4.156574  59.0  2.174995    1  2.457172        8     35   
 1     2.466439  3.533182  53.0  2.303647    1  1.416155        8     90   
 2    -0.204781  4.257076  78.0  2.007251    0  2.501079        8     80   
 3    -1.280169  3.948632  48.0 -1.149298    1  2.477205        8     75   
 4     1.627484  2.562938  71.0  1.795352    0  0.902877        7     90   
 ...        ...       ...   ...       ...  ...       ...      ...    ...   
 9995  2.521690  4.552258  62.0  0.699404    1 -0.150914        8      4   
 9996 -0.384551  4.067641  46.0  1.115856    0  2.329656        7     35   
 9997  2.866007  3.625204  73.0 -0.453329    0  0.179707        9     70   
 9998  3.205736  4.336443  43.0  2.145087    1  1.670624        6      5   
 9999  3.714611  2.465360  78.0 -0.969777    0  1.283133        9     95   
 
           lpsa  
 0     5.548841  
 1     0.832238  
 2     1.885613  
 3     5.21779

In [ ]:
unique,count=np.unique(y,return_counts=True)
unique,count

(array([0, 1]), array([4944, 5056]))

In [ ]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=25)
X_train, y_train = sm.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_res, X_test, y_train_res, y_test = train_test_split(
X_train, y_train, test_size=0.2, random_state=5)

In [ ]:
unique,count=np.unique(y_train,return_counts=True)
unique,count

(array([0, 1]), array([5056, 5056]))

In [ ]:
X_train_res

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
6286,-0.209485,2.379572,55.0,0.062194,0,-0.647267,7,15,4.063400
2682,2.929955,2.836291,52.0,0.901330,0,-0.580578,7,4,3.862216
5674,1.580908,2.483828,70.0,1.818852,0,-0.748635,8,4,3.420369
1353,1.065171,3.231297,64.0,-1.305083,0,-0.539634,9,90,0.841284
5028,-0.926764,3.141126,51.0,-0.591796,1,2.857770,7,15,2.250268
...,...,...,...,...,...,...,...,...,...
3046,-0.114352,2.522258,54.0,1.432630,1,1.284413,7,5,1.970273
9917,-1.002981,4.050404,72.0,-0.494443,0,1.282546,8,0,2.871822
4079,-0.298838,3.518001,54.0,-1.050351,1,-0.593313,8,30,1.045955
2254,3.060966,3.257224,62.0,0.085579,1,1.751875,7,70,0.694583


In [ ]:
X_test

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
7882,-0.219663,3.808598,52.0,-0.149386,0,-1.173706,6,40,2.286634
2712,0.988610,3.040045,64.0,-0.827123,0,1.974362,6,10,1.840884
5887,0.114900,3.825986,55.0,1.091834,0,2.801362,8,5,2.562787
6886,2.324705,3.054295,58.0,1.684833,1,0.403212,9,100,0.076758
9520,-0.704037,3.249040,57.0,0.570058,0,-1.207126,6,100,2.277695
...,...,...,...,...,...,...,...,...,...
5398,1.307816,4.179752,72.0,-0.583676,1,-0.162630,6,30,-0.002621
6305,2.905298,3.477497,59.0,1.343758,1,-0.256351,9,10,3.287828
6938,3.777568,3.343472,65.0,-0.539485,1,2.618915,9,6,1.454175
8690,0.820679,3.045884,61.0,0.576641,1,0.678123,8,35,0.845267


In [ ]:
#ensemble model

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Creating individual classifiers
svm_clf = SVC(kernel='linear', probability=True)
rf_clf = RandomForestClassifier(n_estimators=800, min_samples_split= 2, min_samples_leaf= 1, max_features='sqrt', max_depth= 20, bootstrap =False)

# Creating a voting classifier with soft voting
voting_clf = VotingClassifier(estimators=[('svm', svm_clf), ('rf', rf_clf)], voting='soft')

# Fitting the voting classifier on the training data
voting_clf.fit(X_train_res, y_train_res)

# Making predictions
y_pred = voting_clf.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Model Accuracy:", accuracy)


Ensemble Model Accuracy: 0.8665348492338112


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
dt = DecisionTreeClassifier(max_depth=6, random_state=1)


dt.fit(X_train, y_train)

# Predict test set labels
y_pred = dt.predict(X_test)
score1=dt.score(X_test,y_test)

In [ ]:
score1

0.7777777777777778

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'criterion':['gini','entropy'],
              'max_depth':np.arange(1,21).tolist()[0::2],
              'min_samples_split':np.arange(2,11).tolist()[0::2],
              'max_leaf_nodes':np.arange(3,26).tolist()[0::2]}
g1 = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5, n_jobs=-1)
g1.fit(X_train_res,y_train_res)
g1.best_params_,g1.best_score_

({'criterion': 'gini',
  'max_depth': 9,
  'max_leaf_nodes': 25,
  'min_samples_split': 2},
 0.8162951887126354)

In [ ]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression(fit_intercept=True)
from sklearn.svm import SVC
model = SVC(C=5, kernel='linear')
model.fit(X_train, y_train)
score1= model.score(X_test, y_test)

KeyboardInterrupt: 

In [ ]:
print(score1)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
hyper_params = [{'n_features_to_select': list(range(2,20))}]

In [ ]:
# model_cv = GridSearchCV(estimator = model,
#                         param_grid = hyper_params,
#                         scoring= 'r2',
#                         cv = 8,
#                         verbose = 1,
#                         return_train_score=True)

# # fit the model
# model_cv.fit(X_train, y_train)

Fitting 8 folds for each of 18 candidates, totalling 144 fits


ValueError: Invalid parameter 'n_features_to_select' for estimator LinearRegression(). Valid parameters are: ['copy_X', 'fit_intercept', 'n_jobs', 'positive'].

In [ ]:
model_cv.fit.best_score_

In [ ]:
parameter={'penalty':['l1','l2','elasticnet','linear'],'C':[1,2,3,4,5,10,50],'l1_ratio':[.1,5,.8,.6,1,.3],'n_jobs':[-1]}

In [ ]:
classifier_regressor=GridSearchCV(regression,param_grid=parameter,scoring='accuracy',cv=9)

In [ ]:
svm_parameter={ 'kernel' :['linear', 'poly', 'rbf', 'sigmoid'],'C':[1,2,3,4,5,10,50,60,70,55,45],'max_iter':[25],'decision_function_shape':['ovo', 'ovr'],'random_state':[1,2,5,10,15,30,50]}

In [ ]:
svm_model=GridSearchCV(model,param_grid=svm_parameter,scoring='accuracy',cv=4)

In [ ]:
svm_model.fit(X_train_res.to_numpy(), y_train_res)

In [ ]:
svm_model.best_params_

{'C': 2,
 'decision_function_shape': 'ovo',
 'kernel': 'linear',
 'max_iter': 25,
 'random_state': 1}

In [ ]:
classifier_regressor.fit(X_train_res,y_train_res)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_

GridSearchCV(cv=9, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 10, 50],
                         'l1_ratio': [0.1, 5, 0.8, 0.6, 1, 0.3], 'n_jobs': [-1],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [ ]:
classifier_regressor.best_score_

0.582894629271326

In [ ]:
classifier_regressor.best_params_

{'C': 2,
 'l1_ratio': 0.1,
 'max_iter': 25,
 'n_jobs': -10,
 'penalty': 'l2',
 'solver': 'lbfgs'}

In [ ]:
#pred=classifier_regressor.predict(X_test)
pred1=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,roc_curve,auc
false,true,threshold=roc_curve(y_test,pred1)
roc_auc=auc(false,true)

#score=accuracy_score(y_test,pred)
score1=accuracy_score(y_test,pred1)
#print(score,roc_auc)
score1